This notebook demonstrates how to integrate functionality from SNT with Python through pyimagej.  
The context is the computation and rendering of the convex hull enclosing all axonal/dendritic terminal nodes.  
It requires that the latest version of SNT is installed in your ImageJ/Fiji environment.  

Python dependencies:
- pyimagej
- numpy
- scipy

In [1]:
import sys
# replace with path to your pyimagej directory
sys.path.insert(0, r'C:\Users\cam\Desktop\pyimagej-master\pyimagej')
import imagej
import numpy as np
from scipy.spatial import ConvexHull

# replace with path to your local ImageJ or Fiji installation
# this can be replaced with ij = imagej.init('sc.fiji:fiji')
# once the scijava branch is merged to master
ij = imagej.init(r'C:\Users\cam\Desktop\Fiji.app', headless=False)
from jnius import autoclass, cast


Added 389 JARs to the Java classpath.


In [2]:
# import relevant Java classes
HashSet = autoclass('java.util.HashSet')
PointInImage = autoclass('tracing.util.PointInImage')
MouseLightLoader = autoclass('tracing.io.MouseLightLoader')
Tree = autoclass('tracing.Tree')
TreeAnalyzer = autoclass('tracing.analysis.TreeAnalyzer')
Color = autoclass('org.scijava.util.Colors')
Viewer = autoclass('tracing.viewer.Viewer3D')


In [3]:
def convex_hull(tree):
        """computes the convex hull of the input tree"""
        
        analyzer = TreeAnalyzer(tree)
        # extract the end points from the tree as a java HashSet
        tips_java_set = analyzer.getTips()
        # convert to array to maintain ordering
        tips_java_array = tips_java_set.toArray()

        # convert to python list
        tips_list = [[t.x, t.y, t.z] for t in tips_java_array]

        assert len(tips_list) == tips_java_set.size()

        # Find the convex hull of the end points
        hull = ConvexHull(tips_list)
        
        print("The volume of the convex hull encapsulating all tip points is {} cubic microns".format(
            round(hull.volume, 2)))
        
        print("The area of the hull is {} square microns".format(round(hull.area, 2)))
        print()
        
        # find original PointInImage objects to pass to Viewer3D
        # less costly than re-constructing a new Java object for each vertex
        verts_java = [tips_java_array[i] for i in hull.vertices]
        verts_java = ij.py.to_java(verts_java)
        
        return verts_java


In [4]:
def run():
    
    # fetch swc from MouseLight database by ID
    loader = MouseLightLoader('AA0100')
    if not loader.isDatabaseAvailable():
        print("Could not connect to ML database", "Error")
        return
    if not loader.idExists():
        print("Somewhow the specified id was not found", "Error")
        return

    tree_axon = loader.getTree('axon', None)
    tree_dendrites = loader.getTree('dendrite', None)
    
        
    verts_axon = convex_hull(tree_axon)
    verts_dendrites = convex_hull(tree_dendrites)

    # Visualize the result using SNT Viewer3D
    viewer = Viewer()
    viewer.add(tree_axon)
    viewer.add(tree_dendrites)
    viewer.addSurface(verts_axon, Color.RED, 50)
    viewer.addSurface(verts_dendrites, Color.CYAN, 50)
    viewer.show()
    

In [5]:
run()


The volume of the convex hull encapsulating all tip points is 208791575907.36 cubic microns
The area of the hull is 191199110.74 square microns

The volume of the convex hull encapsulating all tip points is 62095649.03 cubic microns
The area of the hull is 942390.38 square microns

